In [1]:
using ITensors
using LinearAlgebra
using ProgressMeter

global const nsweeps::Int = 100
global const maxdim::Int = 1000
global const cutoff::Float64 = 1e-12
global const noise::Vector{Float64} = [1e-4, 1e-5, 0]

global const observer::DMRGObserver{Float64} = DMRGObserver(energy_tol=1e-6)

global const rho::Float64 = (sqrt(5) - 1) / 2  # 1 / phi
global const rho2::Float64 = (3 - sqrt(5)) / 2;  # 1 / phi^2

ITensors.Strided.disable_threads()
ITensors.Strided.disable_threaded_mul()

In [3]:
function build_HAos(Ad::Matrix{Int64}, s::Int, t::Int, A=50)
    n = size(Ad)[1]
    
    edges = Tuple.(findall(Ad .!= 0))
    N = size(edges)[1]
    
    os = OpSum()
    
    J = zeros(N, N)
    K = zeros(N)
    
    for i in 1:n
        si = findfirst(edges .== ((s, i),))
        is = findfirst(edges .== ((i, s),))
        
        ti = findfirst(edges .== ((t, i),))
        it = findfirst(edges .== ((i, t),))
        
        for j in 1:n
            sj = findfirst(edges .== ((s, j),))
            js = findfirst(edges .== ((j, s),))
            
            # build Hs
            if (sj != nothing) && (si != nothing) && (sj != si)
                os .+= 1, "Z", sj, "Z", si
                J[sj, si] += 1
            end
            
            if (js != nothing) && (is != nothing) && (js != is)
                os .+= 1, "Z", js, "Z", is
                J[js, is] += 1
            end
            
            if (js != nothing) && (si != nothing) && (js != si)
                os .+= -1, "Z", js, "Z", si
                J[js, si] += -1
            end
            
            if (sj != nothing) && (is != nothing) && (sj != is)
                os .+= -1, "Z", sj, "Z", is
                J[sj, is] += -1
            end
            
            tj = findfirst(edges .== ((t, j),))
            jt = findfirst(edges .== ((j, t),))
            
            # build Ht
            if (tj != nothing) && (ti != nothing) && (tj != ti)
                os .+= 1, "Z", tj, "Z", ti
                J[tj, ti] += +1
            end
            
            if (jt != nothing) && (it != nothing) && (jt != it)
                os .+= 1, "Z", jt, "Z", it
                J[jt, it] += +1
            end
            
            if (jt != nothing) && (ti != nothing) && (jt != ti)
                os .+= -1, "Z", jt, "Z", ti
                J[jt, ti] += -1
            end
            
            if (tj != nothing) && (it != nothing) && (tj != it)
                os .+= -1, "Z", tj, "Z", it
                J[tj, it] += -1
            end
            
            # build Hij
            if (i != s) && (i != t)
                if i == j
                    continue
                end
                
                ij = findfirst(edges .== ((i, j),))
                ji = findfirst(edges .== ((j, i),))
                for k in 1:n
                    if i == k
                        continue
                    end
                    ki = findfirst(edges .== ((k, i),))
                    ik = findfirst(edges .== ((i, k),))
                        
                    if (ij != nothing) && (ik != nothing) && (ij != ik)
                        os .+= 1, "Z", ij, "Z", ik
                        J[ij, ik] += 1
                    end

                    if (ji != nothing) && (ki != nothing) && (ji != ki)
                        os .+= 1, "Z", ji, "Z", ki
                        J[ji, ki] += 1
                    end

                    if (ji != nothing) && (ik != nothing) && (ji != ik)
                        os .+= -1, "Z", ji, "Z", ik
                        J[ji, ik] += -1
                    end

                    if (ij != nothing) && (ki != nothing) && (ij != ki)
                        os .+= -1, "Z", ij, "Z", ki
                        J[ij, ki] += -1
                    end
                end
            end
        end
        
        # Hs
        if (si != nothing)
            os .+= -4, "Z", si
            K[si] += -4
        end
        if (is != nothing)
            os .+= +4, "Z", is
            K[is] += +4
        end
        # Ht
        if (ti != nothing)
            os .+= +4, "Z", ti
            K[ti] += +4
        end
        if (it != nothing)
            os .+= -4, "Z", it
            K[it] += -4
        end
    end
    
    J = J .+ transpose(J)
    
    os = OpSum()
    for i in 1:N
        for j in i+1:N
            if J[i, j] != 0
                os += J[i, j], "Z", i, "Z", j
            end
        end
    end
    
    for i in 1:N
        if K[i] != 0
            os += K[i], "Z", i
        end
    end
    
    os *= A / 4
    J *= A / 4
    K *= A / 4
    return os, J, K
end


function build_HBos(N::Int, W, B=1)    
    os = OpSum()
    for j in 1:N
        os .+= B/2*W[j], "Z", j
    end

    return os
end
    
    
function build_Hxos(N::Int)
    os = OpSum()
    for j in 1:N
        os .+= -1, "X", j
    end

    return os
end;

In [5]:
function compute_delta(sites::Vector{Index{Int64}}, N::Int, s::Float64, HA::MPO, HB::MPO, Hx::MPO; 
                       psi0_init=MPS()::MPS, psi1_init=MPS()::MPS)
    H = (1-s)*Hx + s*(HA + HB)
    
    if psi0_init.rlim == 0
        states = [rand()>0.7 ? "Up" : "Dn" for n in 1:N]
        psi0_init = MPS(sites, states)
    end
    
    E0, psi0 = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise=noise,
                    eigsolve_krylovdim=5, outputlevel=0, observer=observer)
    
    if psi1_init.rlim == 0
        states = [rand()>0.7 ? "Up" : "Dn" for n in 1:N]
        psi1_init = MPS(sites, states)
    end

    E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps, maxdim, cutoff, noise=noise,
                    eigsolve_krylovdim=6, outputlevel=0, observer=observer, weight=2)
    
    if E1 > E0
        return E1-E0, psi0, psi1
    else
        return E0-E1, psi1, psi0
    end
end


function find_delta_min(sites::Vector{Index{Int64}}, N::Int, HA::MPO, HB::MPO, Hx::MPO; tol=1e-4::Float64)
    a = 0
    b = 0.4

    h = b - a
    c = a + rho2 * h
    d = a + rho * h
    
    # Required steps to achieve tolerance
    n = trunc(Int, ceil(log(tol / h) / log(rho)))

    yc, psi0, psi1 = compute_delta(sites, N, c, HA, HB, Hx)
    yd, psi2, psi3 = compute_delta(sites, N, d, HA, HB, Hx)
    
    for k in 1:n
        if yc < yd
            b = d
            d = c
            yd = yc
            h = rho * h
            c = a + rho2 * h

            yc, psi0, psi1 = compute_delta(sites, N, c, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
        else
            a = c
            c = d
            yc = yd
            h = rho * h
            d = a + rho * h

            yd, psi0, psi1 = compute_delta(sites, N, d, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
        end
    end
    
    if yc < yd
        sc = (a + d) / 2
    else
        sc = (b + c) / 2
    end
    
    dmin, psi0, psi1 = compute_delta(sites, N, sc, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
    
    return sc, dmin
end;

In [9]:
using PyCall

py"""

import networkx as nx
import random

def adj():
    G = nx.binomial_graph(100, 0.1)
    G = nx.DiGraph(G)

    edges = list(G.edges)
    # random edge choice
    chosen_edge = random.choice(edges)
    G.remove_edge(chosen_edge[0], chosen_edge[1])

    return nx.adjacency_matrix(G).todense()
"""

In [16]:
Ad = py"adj()"

source = trunc(Int, rand()*100)
target = trunc(Int, rand()*100)

edges = Tuple.(findall(Ad .!= 0))
N = size(edges)[1]

os_HA, J, K = build_HAos(Ad, source, target, N)

eigen(J).values[1], N

(-995.0000000000084, 995)

In [ ]:
Ad = [
    0  1  1  1;
    1  0  1  1;
    1  1  0  1;
    1  1  1  0;
]

source = 2
target = 3

edges = Tuple.(findall(Ad .!= 0))
N = size(edges)[1]
sites = siteinds("S=1/2", N)

W = zeros(N)
for i in 1:N
    if reverse(edges[i]) in edges[1:i-1]
        W[i] = W[findfirst(edges .== (reverse(edges[i]),))]
    else
        W[i] = randn() ./ 6 .+ 0.5
    end
end
W[W .> 1] .= 1
W[W .< 0] .= 0

A = N
os_HA, J, K = build_HAos(Ad, source, target, A);
os_HB = build_HBos(N, W)
os_Hx = build_Hxos(N)

HA = MPO(os_HA, sites)
HB = MPO(os_HB, sites)
Hx = MPO(os_Hx, sites)

S = 0.7
delta, psi0, psi1 = compute_delta(sites, N, S, HA, HB, Hx);

In [ ]:
deltaM = zeros(100)
@showprogress for i in 1:100
    delta, sc = compute_delta(sites, N, 0.2, HA, HB, Hx)
    deltaM[i] = delta
end

In [ ]:
using Statistics
mean(deltaM), std(deltaM) / 10

In [ ]:
deltaM = zeros(10)
@showprogress for i in 1:10
    delta, sc = find_delta_min(sites, N, HA, HB, Hx)
    deltaM[i] = delta
end

In [ ]:
using Statistics
mean(deltaM), std(deltaM) / sqrt(10)

In [ ]:
for i in 1:N
    println(edges[i], " ", W[i])
end

In [ ]:
S = 0.2
H = (1-S)*Hx + S*(HA + HB)

states = ["Dn" for n=1:N]
states = [rand()>0.7 ? "Up" : "Dn" for n in 1:N]
psi0_init = MPS(sites, states)

E0, psi0 = dmrg(H, psi0_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=1e-4,
                eigsolve_krylovdim=5, outputlevel=1, observer=observer)

states = ["Dn" for n=1:N]
states = [rand()>0.7 ? "Up" : "Dn" for n in 1:N]
psi1_init = MPS(sites, states)

E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=1e-4,
                eigsolve_krylovdim=6, outputlevel=1, observer=observer, weight=2)

println(E0)
println(E1)

println(inner(psi0', H, psi0))
println(inner(psi1', H, psi1))

In [ ]:
edges[findall(round.(Int, expect(psi0, "Z")) .== 1)]

In [ ]:
inner(psi0', (1-S)*Hx + S*(HA + HB), psi0)

In [ ]:
edges[findall(round.(Int, expect(psi1, "Z")) .== 1)]

In [ ]:
inner(psi1', (1-S)*Hx + S*(HA + HB), psi1)

In [ ]:
state = ["Dn" for i in 1:N]
state[findfirst(edges .== ((2, 6),))] = "Up"
state[findfirst(edges .== ((6, 3),))] = "Up"

psi00 = MPS(sites, state)

inner(psi00', (1-S)*Hx + S*(HA + HB), psi00)

In [ ]:
state = ["Dn" for i in 1:N]
state[findfirst(edges .== ((2, 5),))] = "Up"
state[findfirst(edges .== ((5, 4),))] = "Up"
state[findfirst(edges .== ((4, 3),))] = "Up"
# state[findfirst(edges .== ((4, 9),))] = "Up"

psi11 = MPS(sites, state)

inner(psi11', (1-S)*Hx + S*(HA + HB), psi11)